In [1]:
import ast
import mechanize
from BeautifulSoup import BeautifulSoup
import numpy as np
import pandas as pd
from time import time
from datetime import datetime

In [2]:
with open('account_numbers.txt', 'r') as f:
    account_lines = f.readlines()

In [3]:
account_nums = []

for example in account_lines:
    block = example[:5]
    lot = example[6:10]
    info = ast.literal_eval(example[11:])
    if info != None:
        acct = info[2]
        account_nums.append(acct.encode('ascii','ignore'))
    #print info

In [4]:
pages = {}
times = []

In [5]:
for acct_num in account_nums:

    t0 = time()
    
    br = mechanize.Browser()
    site = 'http://nycserv.nyc.gov/NYCServWeb/WaterChargeHistorySearch.jsp'
    br.open(site)

    br.select_form(name="NycservProtocolForm")

    # Set form data
    br.form['WATER_PAYMENT_HISTORY_ACC_NUMBER'] = acct_num

    br.form.find_control("ServiceName").readonly = False
    br.form.find_control("LinkType").readonly = False
    br.form.find_control("NycservRequest").readonly = False

    br.form['ServiceName'] = 'DEP_WATER_PAYMENT_HISTORY_SEARCH'
    br.form['LinkType'] = 'EMPTY'
    br.form['NycservRequest'] = 'ChannelType=ct/Browser|RequestType=rt/Business|SubSystemType=st/Payments|AgencyType=at/ECB|ServiceName=DEP_WATER_PAYMENT_HISTORY_SEARCH|MethodName=NONE|ParamCount=undefined|WATER_PAYMENT_HISTORY_ACC_NUMBER={}|BOROUGH=1|BLOCK=|LOT=|PageID=WaterChargeHistorySearch|LinkType=EMPTY'.format(acct_num)

    # Submit form
    form_submit = br.submit()
    get_acct_number_html = form_submit.read()
    soup = BeautifulSoup(get_acct_number_html)
    pages[acct_num] = soup
    
    t1 = time()
    total = t1 - t0
    times.append(total)
    
    print 'Current request took: {} seconds'.format(total)
    print 'Average time per request: {} seconds'.format(sum(times)/float(len(times)))
    print 'Num pages scraped: {}'.format(len(pages.keys()))

Current request took: 9.23906493187 seconds
Average time per request: 9.23906493187 seconds
Num pages scraped: 1
Current request took: 7.78889894485 seconds
Average time per request: 8.51398193836 seconds
Num pages scraped: 2
Current request took: 7.57284283638 seconds
Average time per request: 8.20026890437 seconds
Num pages scraped: 3
Current request took: 7.1881070137 seconds
Average time per request: 7.9472284317 seconds
Num pages scraped: 4
Current request took: 7.95972895622 seconds
Average time per request: 7.94972853661 seconds
Num pages scraped: 5
Current request took: 7.69737100601 seconds
Average time per request: 7.90766894817 seconds
Num pages scraped: 6
Current request took: 8.89435195923 seconds
Average time per request: 8.04862366404 seconds
Num pages scraped: 7
Current request took: 13.1047239304 seconds
Average time per request: 8.68063619733 seconds
Num pages scraped: 8
Current request took: 18.3859269619 seconds
Average time per request: 9.75900183784 seconds
Num pa

In [6]:
dfs = {}

In [7]:
for acct_num, soup in pages.items():

    form = soup.findAll('form')
    tables = soup.findAll('table')
    payments = tables[4]

    rows = payments.findAll('tr')

    header = rows[0]
    cols = header.findAll('td')

    names = []

    for col in cols:
        raw = col.contents[0]
        if '&nbsp;' in raw:
            raw = raw.replace('&nbsp;', '')
        raw = raw.encode('ascii','ignore')
        names.append(raw.lstrip('\r\n\t'))

    all_rows = []

    data = rows[1:]

    for row in data:
        data_cols = row.findAll('td')
        row_data = []

        for col in data_cols:
            raw = col.contents[0]
            if '&nbsp;' in raw:
                raw = raw.replace('&nbsp;', '')
            raw = raw.encode('ascii','ignore')
            row_data.append(raw.rstrip('\n'))

        all_rows.append(row_data)

    all_rows = np.array(all_rows)

    dfs[acct_num] = pd.DataFrame(all_rows[:,1:], columns=names[1:] )

In [8]:
for acct_num, df in dfs.items():
    try:
        df['Amount'] = df['Amount'].apply(lambda x: x.replace(',', ''))
        df['Amount'] = df['Amount'].astype(float)
        df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
    except Exception as e:
        print 'error with dataframe:'
        print e
        print df

In [37]:
from ipywidgets.widgets import Select
from IPython.display import display, clear_output

In [24]:
with open('account_numbers.txt', 'r') as f:
    account_lines = f.readlines()
    
lot_to_acct = {}

for example in account_lines:
    block = example[:5]
    lot = example[6:10]
    info = ast.literal_eval(example[11:])
    if info != None:
        acct = info[2]
        lot_to_acct[lot] = acct.encode('ascii','ignore')

In [47]:
lot = Select(description='Choose a lot:', height='200px', options=lot_to_acct.keys())

In [51]:
def display_df():
    clear_output()
    acct_num = lot_to_acct[lot.value]
    print 'Total: {}'.format(dfs[acct_num]['Amount'].sum())
    print
    display(dfs[acct_num])

lot.on_trait_change(display_df, 'value')

In [52]:
display(lot)

Total: 3537.84



,Date,Transaction Type,Amount
0,2014-11-10,Payment,680.39
1,2015-02-17,Payment,776.22
2,2015-05-28,Payment,709.14
3,2015-08-06,Payment,641.87
4,2015-11-05,Payment,730.22


In [58]:
df.to_csv()

',Date,Transaction Type,Amount\n0,2014-11-19,Payment,574.98\n1,2014-12-31,Payment,249.16\n2,2015-04-22,Payment,507.9\n3,2015-07-15,Payment,354.57\n4,2015-08-19,Payment,195.36\n5,2015-09-16,Payment,148.02\n6,2015-10-20,Payment,177.62\n7,2015-11-18,Payment,98.54\n'

In [61]:
# Persist all data

In [72]:
with open('account_numbers.txt', 'r') as f:
    account_lines = f.readlines()
    
with open('payments.txt', 'w') as f:
    
    #f.write('block,lot,account,name,address,total_payment,\ndf')
    
    for example in account_lines:
        block = example[:5]
        lot = example[6:10]
        info = ast.literal_eval(example[11:])
        if info != None:
            name = info[0].encode('ascii','ignore')
            address = info[1].encode('ascii','ignore')
            address = address.replace(',', ' ')
            address = address.replace('\n', ' ')
            acct_num = info[2].encode('ascii','ignore')
            df = dfs[acct_num]
            csv = df.to_csv()
            total = df['Amount'].sum()
            line = '{},{},{},{},{},{},\n{}\n'.format(block, lot, acct_num, name, address, total, csv)
            f.write(line)